In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip -q install langchain langchain_community pypdf chromadb langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.9 MB/s eta 0

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = "YOUR_API_KEY"

## **RAG의 핵심, 문서 검색기 Retriever**

### **사용자의 쿼리를 재해석하여 검색하다, MultiQueryRetriever**

**Chroma DB에 문서 벡터 저장**

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
#헌법 PDF 파일 로드
loader = PyPDFLoader(r"/content/drive/MyDrive/마소캠퍼스/대한민국 헌법.pdf")
pages = loader.load_and_split()

#PDF 파일을 500자 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(pages)

#ChromaDB에 청크들을 벡터 임베딩으로 저장(OpenAI 임베딩 모델 활용)
db = Chroma.from_documents(docs, OpenAIEmbeddings(model = 'text-embedding-3-small'))

**질문을 여러 버전으로 재해석하여 Retriever에 활용**

In [ ]:
pip install langchain_openai

In [ ]:
#```Chroma DB에 대한민국 헌법 PDF 임베딩 변환 및 저장하는 과정은 위 셀에 있습니다```
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI

#질문 문장 question으로 저장
question = "국회의원의 의무는 무엇이 있나요?"
#여러 버전의 질문으로 변환하는 역할을 맡을 LLM 선언
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125",
                 temperature = 0)
#MultiQueryRetriever에 벡터DB 기반 Retriever와 LLM 선언
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=db.as_retriever(), llm=llm
)

# 여러 버전의 문장 생성 결과를 확인하기 위한 로깅 과정
import logging
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

#여러 버전 질문 생성 결과와 유사 청크 검색 개수 출력
unique_docs = retriever_from_llm.invoke(input=question)
len(unique_docs)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. 국회의원이 가져야 하는 책임은 어떤 것들이 있습니까?', '2. 국회의원은 어떤 의무를 부담해야 하나요?', '3. 국회의원이 수행해야 하는 역할에는 어떤 것들이 포함되어 있나요?']


6

In [ ]:
unique_docs

[Document(metadata={'page': 9, 'source': '/content/drive/MyDrive/마소캠퍼스/대한민국 헌법.pdf'}, page_content='한 동의권을 가진다.\n \n제61조 ①국회는 국정을 감사하거나 특정한 국정사안에 대하여 조사할 수 있으며, 이에 필요한\n서류의 제출 또는 증인의 출석과 증언이나 의견의 진술을 요구할 수 있다.\n②국정감사 및 조사에 관한 절차 기타 필요한 사항은 법률로 정한다.\n \n제62조 ①국무총리ㆍ국무위원 또는 정부위원은 국회나 그 위원회에 출석하여 국정처리상황을\n보고하거나 의견을 진술하고 질문에 응답할 수 있다.\n②국회나 그 위원회의 요구가 있을 때에는 국무총리ㆍ국무위원 또는 정부위원은 출석ㆍ답변\n하여야 하며, 국무총리 또는 국무위원이 출석요구를 받은 때에는 국무위원 또는 정부위원으로\n하여금 출석ㆍ답변하게 할 수 있다.\n \n제63조 ①국회는 국무총리 또는 국무위원의 해임을 대통령에게 건의할 수 있다.\n②제1항의 해임건의는 국회재적의원 3분의 1 이상의 발의에 의하여 국회재적의원 과반수의\n찬성이 있어야 한다.'),
 Document(metadata={'page': 7, 'source': '/content/drive/MyDrive/마소캠퍼스/대한민국 헌법.pdf'}, page_content='제43조 국회의원은 법률이 정하는 직을 겸할 수 없다.\n \n제44조 ①국회의원은 현행범인인 경우를 제외하고는 회기중 국회의 동의없이 체포 또는 구금되\n지 아니한다.\n②국회의원이 회기전에 체포 또는 구금된 때에는 현행범인이 아닌 한 국회의 요구가 있으면\n회기중 석방된다.\n \n제45조 국회의원은 국회에서 직무상 행한 발언과 표결에 관하여 국회외에서 책임을 지지 아니한\n다.\n \n제46조 ①국회의원은 청렴의 의무가 있다.\n②국회의원은 국가이익을 우선하여 양심에 따라 직무를 행한다.\n③국회의원은 그 지위를 남용하여 국

### **컨텍스트 재정렬, Long-Context Reorder**

**[Long-Context Reorder 없이 유사 문서 출력]**

In [ ]:
#Chroma dimension 관련 에러 발생 시 실행
# Chroma().delete_collection()

In [ ]:
pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 6.9 MB/s eta 0:00:00


In [ ]:
from langchain.prompts import PromptTemplate
from langchain_community.document_transformers import (
    LongContextReorder,
)
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAI

texts = [
    "바스켓볼은 훌륭한 스포츠입니다.",
    "플라이 미 투 더 문은 제가 가장 좋아하는 노래 중 하나입니다.",
    "셀틱스는 제가 가장 좋아하는 팀입니다.",
    "이것은 보스턴 셀틱스에 관한 문서입니다."
    "저는 단순히 영화 보러 가는 것을 좋아합니다",
    "보스턴 셀틱스가 20점차로 이겼어요",
    "이것은 그냥 임의의 텍스트입니다.",
    "엘든 링은 지난 15 년 동안 최고의 게임 중 하나입니다.",
    "L. 코넷은 최고의 셀틱스 선수 중 한 명입니다.",
    "래리 버드는 상징적인 NBA 선수였습니다.",
]
# Chroma Retriever 선언(10개의 유사 문서 출력)
retriever = FAISS.from_texts(texts, OpenAIEmbeddings(model = 'text-embedding-3-small')).as_retriever(
    search_kwargs={"k": 10}
)
query = "셀틱에 대해 설명해줘"

# 유사도 기준으로 검색 결과 출력
docs = retriever.invoke(query)
for i in docs:
  print(i.page_content)

셀틱스는 제가 가장 좋아하는 팀입니다.
보스턴 셀틱스가 20점차로 이겼어요
이것은 보스턴 셀틱스에 관한 문서입니다.저는 단순히 영화 보러 가는 것을 좋아합니다
L. 코넷은 최고의 셀틱스 선수 중 한 명입니다.
이것은 그냥 임의의 텍스트입니다.
바스켓볼은 훌륭한 스포츠입니다.
엘든 링은 지난 15 년 동안 최고의 게임 중 하나입니다.
플라이 미 투 더 문은 제가 가장 좋아하는 노래 중 하나입니다.
래리 버드는 상징적인 NBA 선수였습니다.


**[Long-Context Reorder 활용하여 유사 문서 출력]**

In [ ]:
#LongContextReorder 선언
reordering = LongContextReorder()
#검색된 유사문서 중 관련도가 높은 문서를 맨앞과 맨뒤에 재정배치
reordered_docs = reordering.transform_documents(docs)
for i in reordered_docs:
  print(i.page_content)

셀틱스는 제가 가장 좋아하는 팀입니다.
이것은 보스턴 셀틱스에 관한 문서입니다.저는 단순히 영화 보러 가는 것을 좋아합니다
이것은 그냥 임의의 텍스트입니다.
엘든 링은 지난 15 년 동안 최고의 게임 중 하나입니다.
래리 버드는 상징적인 NBA 선수였습니다.
플라이 미 투 더 문은 제가 가장 좋아하는 노래 중 하나입니다.
바스켓볼은 훌륭한 스포츠입니다.
L. 코넷은 최고의 셀틱스 선수 중 한 명입니다.
보스턴 셀틱스가 20점차로 이겼어요


### **필요없는 문서는 삭제, Contextual Compression**

In [ ]:
# Helper function for printing docs
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveJsonSplitter

loader = PyPDFLoader(r"/content/drive/MyDrive/마소캠퍼스/대한민국 헌법.pdf")
pages = loader.load_and_split()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(pages)
db = FAISS.from_documents(docs, OpenAIEmbeddings(model = 'text-embedding-3-small'))
retriever =db.as_retriever()

docs = retriever.invoke("대통령의 임기는?")
pretty_print_docs(docs)

Document 1:

제70조 대통령의 임기는 5년으로 하며, 중임할 수 없다.
 
제71조 대통령이 궐위되거나 사고로 인하여 직무를 수행할 수 없을 때에는 국무총리, 법률이 정
한 국무위원의 순서로 그 권한을 대행한다.
 
제72조 대통령은 필요하다고 인정할 때에는 외교ㆍ국방ㆍ통일 기타 국가안위에 관한 중요정책
을 국민투표에 붙일 수 있다.
 
제73조 대통령은 조약을 체결ㆍ비준하고, 외교사절을 신임ㆍ접수 또는 파견하며, 선전포고와 강
화를 한다.
----------------------------------------------------------------------------------------------------
Document 2:

법제처                                                            11                                                       국가법령정보센터
「대한민국헌법」 
③탄핵소추의 의결을 받은 자는 탄핵심판이 있을 때까지 그 권한행사가 정지된다.
④탄핵결정은 공직으로부터 파면함에 그친다. 그러나, 이에 의하여 민사상이나 형사상의 책임
이 면제되지는 아니한다.
 
                    제4장 정부
                       제1절 대통령
 
제66조 ①대통령은 국가의 원수이며, 외국에 대하여 국가를 대표한다.
②대통령은 국가의 독립ㆍ영토의 보전ㆍ국가의 계속성과 헌법을 수호할 책무를 진다.
③대통령은 조국의 평화적 통일을 위한 성실한 의무를 진다.
④행정권은 대통령을 수반으로 하는 정부에 속한다.
 
제67조 ①대통령은 국민의 보통ㆍ평등ㆍ직접ㆍ비밀선거에 의하여 선출한다.
----------------------------------------------------------------------------------------------------
Document 3:

②대통령은 국가의 안위에 관계되는 중대

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_openai import OpenAI

llm = ChatOpenAI(model ='gpt-4o-mini', temperature=0)
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(
    "대통령의 임기는?"
)
pretty_print_docs(compressed_docs)

Document 1:

제70조 대통령의 임기는 5년으로 하며, 중임할 수 없다.
----------------------------------------------------------------------------------------------------
Document 2:

제67조 ①대통령은 국민의 보통ㆍ평등ㆍ직접ㆍ비밀선거에 의하여 선출한다.
----------------------------------------------------------------------------------------------------
Document 3:

제68조 ①대통령의 임기가 만료되는 때에는 임기만료 70일 내지 40일전에 후임자를 선거한다.


In [ ]:
from langchain.retrievers.document_compressors import LLMChainFilter

_filter = LLMChainFilter.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=_filter, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(
    "대통령의 임기는?"
)
pretty_print_docs(compressed_docs)

Document 1:

제70조 대통령의 임기는 5년으로 하며, 중임할 수 없다.
 
제71조 대통령이 궐위되거나 사고로 인하여 직무를 수행할 수 없을 때에는 국무총리, 법률이 정
한 국무위원의 순서로 그 권한을 대행한다.
 
제72조 대통령은 필요하다고 인정할 때에는 외교ㆍ국방ㆍ통일 기타 국가안위에 관한 중요정책
을 국민투표에 붙일 수 있다.
 
제73조 대통령은 조약을 체결ㆍ비준하고, 외교사절을 신임ㆍ접수 또는 파견하며, 선전포고와 강
화를 한다.
----------------------------------------------------------------------------------------------------
Document 2:

법제처                                                            11                                                       국가법령정보센터
「대한민국헌법」 
③탄핵소추의 의결을 받은 자는 탄핵심판이 있을 때까지 그 권한행사가 정지된다.
④탄핵결정은 공직으로부터 파면함에 그친다. 그러나, 이에 의하여 민사상이나 형사상의 책임
이 면제되지는 아니한다.
 
                    제4장 정부
                       제1절 대통령
 
제66조 ①대통령은 국가의 원수이며, 외국에 대하여 국가를 대표한다.
②대통령은 국가의 독립ㆍ영토의 보전ㆍ국가의 계속성과 헌법을 수호할 책무를 진다.
③대통령은 조국의 평화적 통일을 위한 성실한 의무를 진다.
④행정권은 대통령을 수반으로 하는 정부에 속한다.
 
제67조 ①대통령은 국민의 보통ㆍ평등ㆍ직접ㆍ비밀선거에 의하여 선출한다.
----------------------------------------------------------------------------------------------------
Document 3:

②제1항의 선거에 있어서 최고득표자가 

In [ ]:
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain_community.document_transformers import EmbeddingsRedundantFilter
from langchain.retrievers.document_compressors import EmbeddingsFilter
from langchain_text_splitters import CharacterTextSplitter

llm = ChatOpenAI(model ='gpt-4o-mini', temperature=0)

splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0, separator=". ")
compressor = LLMChainExtractor.from_llm(llm)
relevant_filter = EmbeddingsFilter(embeddings=OpenAIEmbeddings(model = 'text-embedding-3-small'), similarity_threshold=0.4)

pipeline_compressor = DocumentCompressorPipeline(
    transformers=[splitter, relevant_filter, compressor]
)

In [ ]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=pipeline_compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(
    "대통령의 임기는?"
)
pretty_print_docs(compressed_docs)

Document 1:

제70조 대통령의 임기는 5년으로 하며, 중임할 수 없다.
----------------------------------------------------------------------------------------------------
Document 2:

제68조 ①대통령의 임기가 만료되는 때에는 임기만료 70일 내지 40일전에 후임자를 선거한다.


### **가상 문서로 유사 문서 탐색, HyDE**(Hypothetical Document Embedding)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

system = """
당신은 LangChain, LangGraph, LangServe, LangSmith라는 LLM 기반 애플리케이션을 구축하기 위한 일련의 소프트웨어에 대한 전문가입니다.

LangChain은 LLM 애플리케이션을 구축하기 위해 쉽게 구성할 수 있는 대규모 통합 세트를 제공하는 Python 프레임워크입니다.
LangGraph는 상태 저장, 멀티 액터 LLM 애플리케이션을 쉽게 구축할 수 있는 LangChain 위에 구축된 Python 패키지입니다.
LangServe는 REST API로 LangChain 애플리케이션을 쉽게 배포할 수 있는 LangChain 위에 구축된 Python 패키지입니다.
LangSmith는 LLM 애플리케이션 추적 및 테스트를 쉽게 할 수 있는 플랫폼입니다.

사용자 질문에 최선을 다해 답변하세요. 사용자 질문에 대한 튜토리얼을 작성하는 것처럼 답변하세요.
"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
qa_no_context = prompt | llm | StrOutputParser()

In [ ]:
answer = qa_no_context.invoke(
    {
        "question": "Chain을 구축할 때 멀티모달 모델을 활용하는 방법과 Chain을 REST API로 전환하는 방법"
    }
)
print(answer)

Chain을 구축할 때 멀티모달 모델을 활용하고 이를 REST API로 전환하는 방법에 대해 단계별로 설명하겠습니다. 멀티모달 모델은 텍스트, 이미지, 오디오 등 다양한 형태의 데이터를 처리할 수 있는 모델을 의미합니다. 이를 LangChain과 LangServe를 활용하여 구현할 수 있습니다.

### 1단계: 멀티모달 모델 선택 및 설정

먼저, 멀티모달 모델을 선택해야 합니다. 예를 들어, OpenAI의 CLIP 모델이나 DALL-E 모델을 사용할 수 있습니다. 이 모델들은 텍스트와 이미지를 함께 처리할 수 있는 기능을 제공합니다.

```python
from langchain import OpenAI

# OpenAI 모델 초기화
model = OpenAI(model_name="gpt-3.5-turbo")  # 텍스트 모델
```

### 2단계: Chain 구축

Chain을 구축하기 위해 LangChain의 다양한 구성 요소를 사용할 수 있습니다. 예를 들어, 텍스트와 이미지를 입력으로 받아 처리하는 Chain을 만들 수 있습니다.

```python
from langchain.chains import SimpleSequentialChain
from langchain.prompts import PromptTemplate

# 텍스트와 이미지를 처리하는 프롬프트 템플릿 생성
text_prompt = PromptTemplate(input_variables=["text"], template="이 텍스트에 대한 설명: {text}")
image_prompt = PromptTemplate(input_variables=["image"], template="이 이미지를 설명해 주세요: {image}")

# Chain 구성
chain = SimpleSequentialChain(chains=[text_prompt, image_prompt])
```

### 3단계: Chain 테스트

구축한 Chain을 테스트하여 올바르게 작동하는지 확인합니다.

```pytho

In [ ]:
from langchain_core.runnables import RunnablePassthrough

hyde_chain = RunnablePassthrough.assign(hypothetical_document=qa_no_context)

hyde_chain.invoke(
    {
        "question": "Chain을 구축할 때 멀티모달 모델을 활용하는 방법과 Chain을 REST API로 전환하는 방법"
    }
)

{'question': 'Chain을 구축할 때 멀티모달 모델을 활용하는 방법과 Chain을 REST API로 전환하는 방법',
 'hypothetical_document': 'Chain을 구축할 때 멀티모달 모델을 활용하고, 이를 REST API로 전환하는 방법에 대해 단계별로 설명하겠습니다. 이 튜토리얼에서는 LangChain과 LangServe를 사용하여 멀티모달 Chain을 구축하고 이를 REST API로 배포하는 과정을 다룹니다.\n\n### 1단계: 멀티모달 모델 설정\n\n멀티모달 모델은 텍스트, 이미지, 오디오 등 다양한 형태의 데이터를 처리할 수 있는 모델입니다. LangChain을 사용하여 멀티모달 Chain을 구축하기 위해 필요한 라이브러리를 설치합니다.\n\n```bash\npip install langchain\npip install transformers  # Hugging Face Transformers 라이브러리\n```\n\n### 2단계: 멀티모달 Chain 구축\n\n이제 멀티모달 Chain을 구축해 보겠습니다. 예를 들어, 텍스트와 이미지를 입력으로 받아서 텍스트에 대한 설명을 생성하는 Chain을 만들어 보겠습니다.\n\n```python\nfrom langchain import Chain\nfrom langchain.llms import OpenAI\nfrom langchain.prompts import PromptTemplate\nfrom langchain.chains import SimpleSequentialChain\n\n# 텍스트와 이미지를 처리할 LLM 설정\nllm = OpenAI(model="gpt-3.5-turbo")\n\n# 텍스트 프롬프트 템플릿\ntext_prompt = PromptTemplate(\n    input_variables=["text"],\n    template="이 텍스트에 대한 설명을 작성하세요: {text}"\n)\n\n# 이미지 프롬프트 템플릿 (예시로 텍스트로 설명)\nimag